### Sending Exceptions to Generators

#### Sending things to coroutines

Use .send(data) -> sends data to coroutine  
Use .close() -> sends (throws) a GeneratorExit exception to coroutine

We can also "send" any exception to the coroutine -> throwing an exception to the coroutine

Use .throw(exception) -> the exception is raised at the point where the coroutine is suspended

#### How throw() is handled

- generator does not catch the exception (does nothing) -> exception propagates to caller
- generator catches the exception, and does something -> yields a value or -> exits (returns) or -> raises a different exception

#### Catch and yield

- generator catches the exception
- handles and silences the exception
- yields a value
- generator is now suspended

The yielded value is the return value of the .throw() method

#### Catch and exit

- generator catches the exception
- generator exits(returns)
- caller receives a StopIteration exception -> generator is now closed. This is the same as calling next() or send() to a generator that returns instead of yielding
- can think of throw() as same thing as send(), but causes an exception to be sent instead of plain data

#### Catch and raise different exception

- generator catches the exception
- generator handles the exception and raises another exception
- new exception propagates to caller -> generator now closed

#### close() vs throw()

- close() -> GeneratorExit exception is raised inside generator
- can we just call gen.throw(GeneratorExit()) ?

yes, but... with close(), Python expects the GeneratorExit, or StopIteratioon exceptions to propagate, and silences it for the caller

If we use throw() instead, the GeneratorExit exception is raised inside the caller context (if the generator lets it)

### Code Examples

In [3]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    finally:
        print('exception must have happened')

In [4]:
g = gen()

In [5]:
next(g)

In [6]:
g.send('hello')

hello


In [7]:
g.throw(ValueError, 'custom message')

exception must have happened


ValueError: custom message

In [8]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    except ValueError:
        print('received a value error...')
    finally:
        print('exception must have happened')

In [9]:
g = gen()

In [10]:
next(g)

In [11]:
g.send('hello')

hello


In [12]:
g.throw(ValueError, 'custom message')

received a value error...
exception must have happened


StopIteration: 

In [13]:
from inspect import getgeneratorstate

In [14]:
def gen():
    while True:
        try:
            received = yield
            print(received)
        except ValueError as ex:
            print('Value error received:', str(ex))

In [15]:
g = gen()
next(g)

In [16]:
g.send('hello')

hello


In [17]:
g.throw(ValueError, 'custom message')

Value error received: custom message


In [18]:
getgeneratorstate(g)

'GEN_SUSPENDED'

In [19]:
g.send('Python')

Python


In [20]:
def gen():
    while True:
        received = yield
        print(received)

In [21]:
g = gen()
next(g)

In [22]:
g.send('hello')

hello


In [23]:
g.throw(ValueError, 'custom message')

ValueError: custom message

In [25]:
getgeneratorstate(g)

'GEN_CLOSED'

In [26]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    except ValueError as ex:
        print('ValueError received', str(ex))
        return None

In [27]:
g = gen()
next(g)
g.send('hello')

hello


In [28]:
g.throw(ValueError, 'custom message')

ValueError received custom message


StopIteration: 

In [29]:
getgeneratorstate(g)

'GEN_CLOSED'

In [30]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    except ValueError as ex:
        print('ValueError received....', str(ex))
        raise ZeroDivisionError("not really!")

In [31]:
g = gen()
next(g)
g.send('hello')

hello


In [32]:
g.throw(ValueError, 'custom message')

ValueError received.... custom message


ZeroDivisionError: not really!

In [36]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    except ValueError as ex:
        print('ValueError received....', str(ex))
        raise ZeroDivisionError("not really!") from None # from None silences the ValueError in the traceback

In [37]:
g = gen()
next(g)
g.send('hello')

hello


In [38]:
g.throw(ValueError, 'custom message')

ValueError received.... custom message


ZeroDivisionError: not really!

In [40]:
class CommitException(Exception):
    pass

class RollbackException(Exception):
    pass

def write_to_db():
    print('opening database connection...')
    print('starting transaction...')
    try:
        while True:
            try:
                data = yield
                print('writing data to database...', data)
            except CommitException:
                print('commiting transaction...')
                print('opening next transaction...')
            except RollbackException:
                print('abort transaction...')
                print('opening next transaction...')
    finally:
        print('generator closing...')
        print('abort transaction...')
        print('closing database connection...')    

In [41]:
sql = write_to_db()

In [42]:
next(sql)

opening database connection...
starting transaction...


In [43]:
sql.send(100)

writing data to database... 100


In [44]:
sql.throw(CommitException)

commiting transaction...
opening next transaction...


In [45]:
sql.send(200)

writing data to database... 200


In [46]:
sql.throw(RollbackException)

abort transaction...
opening next transaction...


In [47]:
sql.close()

generator closing...
abort transaction...
closing database connection...


In [48]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    finally:
        print('closing down')

In [49]:
g = gen()

In [50]:
next(g)

In [51]:
g.close()

closing down


In [52]:
g = gen()
next(g)

In [53]:
g.throw(GeneratorExit)

closing down


GeneratorExit: 

In [54]:
def gen():
    try:
        while True:
            received = yield
            print(received)
    except GeneratorExit:
        print('received GeneratorExit...')
    finally:
        print('closing down')

In [55]:
g = gen()
next(g)

In [56]:
g.close()

received GeneratorExit...
closing down


In [57]:
g = gen()
next(g)

In [58]:
g.throw(GeneratorExit)

received GeneratorExit...
closing down


StopIteration: 